# Weighting, Matching, or Regression?

*Comparing different methods to condition causal inference on covariates*

INTRO

## Example

Assume we were a firm interested in understanding the effect of $D$ on $Y$. Unfortunately, we cannot randomize $D$. Unfortunately we cannot randomize $D$. 

Assume the data generating process can be represented with the following [**Directed Acyclic Graph (DAG)**](https://towardsdatascience.com/b63dc69e3d8c). If you are not familiar with DAGs, I have written a short [introduction here](https://towardsdatascience.com/b63dc69e3d8c).

```mermaid
flowchart LR
classDef included fill:#DCDCDC,stroke:#000000,stroke-width:2px;
classDef excluded fill:#ffffff,stroke:#000000,stroke-width:2px;
classDef unobserved fill:#ffffff,stroke:#000000,stroke-width:2px,stroke-dasharray: 5 5;

X1((gender))
X2((age))
X3((income))
D((D))
Y((Y))

D --> Y
X1 --> D
X2 --> D
X3 --> D
X1 --> Y
X2 --> Y
X3 --> Y

class D,Y included;
class X1,X2,X3 excluded;
```

## Conclusion

TBD

### References

TBD

### Related Articles

- [Understanding The Frisch-Waugh-Lovell Theorem](https://towardsdatascience.com/59f801eb3299)
- [Understanding Contamination Bias](https://towardsdatascience.com/58b63d25d2ef)
- [DAGs and Control Variables](https://towardsdatascience.com/b63dc69e3d8c)

### Code

You can find the original Jupyter Notebook here:

https://github.com/matteocourthoud/Blog-Posts/blob/main/notebooks/ipw.ipynb